# End Pointing of Speech signal

* Based on [Paper by Rabiner and Sambur](http://ieeexplore.ieee.org/stamp/stamp.jsp?arnumber=6778857)

In [93]:
%matplotlib inline
from scipy import signal
from scipy.io.wavfile import write,read
import matplotlib.pyplot as plt
import math
import numpy as np
import IPython
from numpy import convolve
from scipy.signal import hamming
from scipy.ndimage.interpolation import shift
from scipy.linalg import toeplitz,inv
from scipy.fftpack import fft,fftshift,ifft

# Returns ndarray corresponding to voiced part in filename wav file
def endpointer(filename,Fs):
    Ts=1/Fs
    read_wav = read(filename+'.wav')
    inp=np.array(read_wav[1],dtype='float64')
    plt.rcParams["figure.figsize"] = (18,10)
    #plt.title("Raw Input")
    #plt.plot(inp,'blue')
    
    #------------------------------------------------------------------------------
    #Compute Avg magnitude (Short Time), with window size N/20 (0-9 spoken twice)
    W=50
    pad_zeros=np.zeros(W)
    zp_inp_t=np.append(pad_zeros,inp)
    zp_inp=np.append(zp_inp_t,pad_zeros)
    st_avg=[np.sum(abs(zp_inp[i-W:i+W])) for i in range(W,zp_inp.size-W)]
    #plt.plot(st_avg,'red')

    #Compute ZCR
    st_zcr=[np.where(np.diff(np.signbit(inp[i-W:i+W])))[0].size for i in range(W,zp_inp.size-W)]
    #plt.plot(st_zcr,'green')
    
    #Compute IZCT assuming 100ms silence in start
    num_samp_in_sil=int(Fs*0.1)
    sil_inp=inp[0:num_samp_in_sil]
    sil_inp_t=np.append(pad_zeros,sil_inp)
    sil_inp_wpad=np.append(sil_inp_t,pad_zeros)
    #Compute ZCR
    sil_zcr=[np.where(np.diff(np.signbit(sil_inp_wpad[i-W:i+W])))[0].size for i in range(W,sil_inp_wpad.size-W)]
    sil_avg=[np.sum(np.abs(sil_inp_wpad[i-W:i+W])) for i in range(W,sil_inp_wpad.size-W)]
    
    
    sigma_izc=np.std(sil_zcr)
    mean_izc=np.mean(sil_zcr)
    IF=25
    
    IZCT=min(IF,mean_izc+(2*sigma_izc))
    #print (mean_izc+(2*sigma_izc))
    
    #Compute Peak Energy Imax and Silence Energy Imin
    Imx=np.amax(st_avg)
    Imn=np.amax(sil_avg)
    #print(Imn)
    #Compute ITL and ITU
    I1=0.03*(Imx-Imn)+Imn
    I2=4*Imn
    ITL=min(I1,I2)
    print(ITL)
    ITU=10*ITL
    
    #Search Fwd
    for i in range(8,9):
        if(i==0):
            start_index=num_samp_in_sil+1
        else:
            start_index=int((i/20)*inp.size)
        end_index=int(((i+1)/20)*inp.size)-1
        print(start_index)
        print (end_index)
        zero_st,zero_fi=search(st_avg,st_zcr,start_index,end_index,IZCT,ITU,ITL)
        print (zero_st)
        print (zero_fi)
        IPython.display.Audio(np.array(inp[zero_st:zero_fi]),rate=8000)
        write('test.wav', 8000, np.array(inp[zero_st:zero_fi]).astype(np.dtype('i2')))
        plt.plot(inp[zero_st:zero_fi])
        
endpointer('./Data/Digits_male_8Khz/zero_to_nine_Hitesh1',8000)
    

22721.94
90703
102040
0
0


ValueError: zero-size array to reduction operation maximum which has no identity

In [76]:
def search(st_avg,st_zcr,start_index,end_index,IZCT,ITU,ITL):
    N1=0
    N2=0
    m=start_index    
    #Fwd Search
    while(m<end_index):
        flag=0
        contender=0
        i=0
        #print(st_avg[m]-ITL)
        if(st_avg[m] >= ITL):
            i=m
            while(True):
                if(st_avg[i]<ITL):
                    flag=1
                    break
                if(st_avg[i]>=ITU):
                    contender=i
                    if(i==m):
                        contender=contender-1
                    flag=2
                    break
                else: 
                    i=i+1
            
            if(flag==2):              
                M1=0
                last_index=0
                for j in range(1,26):
                    if(st_zcr[contender-j]>IZCT):
                        M1=M1+1
                        last_index=j
                if (M1 > 2):
                    N1=contender-last_index
                    break
                else:
                    N1=contender
                    break
        if(flag==1):
            m=i+1
            continue
        if(flag==0):
            m=m+1
            continue

                
    #Bkwd Search
    m=end_index
    while(m>=start_index):
        flag=0
        contender=0
        i=0
        if(st_avg[m] >= ITL):
            i=m
            while(True):
                if(st_avg[i]<ITL):
                    flag=1
                    break
                if(st_avg[i]>=ITU):
                    contender=i
                    if(i==m):
                        contender=contender+1
                    flag=2
                    break
                else: 
                    i=i-1
            
            if(flag==2):
                M1=0
                last_index=0
                for j in range(1,26):
                    if(st_zcr[contender+j]>IZCT):
                        M1=M1+1
                        last_index=j
                if (M1 > 2):
                    N2=contender+last_index
                    break
                else:
                    N2=contender
                    break
                
        if(flag==1):
            m=i-1
        if(flag==0):
            m=m-1
            
    return (N1,N2)
            
    
 